In [ ]:
# %load_ext autoreload
# %autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
import copy
import sys
from importlib import reload
from typing import Dict, List, Tuple

import addict
import celeri
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import collections
from tqdm.notebook import tqdm

plt.rcParams["text.usetex"] = (
    False  # Plotting the global model is much much faster with tex fonts turned off
)

# Read in data files, create storage dictionaries, and do basic processing

In [ ]:
# Western North America example
config_file_name = "../data/config/western_north_america_config.json"

config = celeri.get_config(config_file_name)

# Adjust mesh values
config.mesh_params[0].smoothing_weight = 1e16
config.mesh_params[0].top_slip_rate_constraint = 0
config.mesh_params[0].bot_slip_rate_constraint = 0
config.mesh_params[0].side_slip_rate_constraint = 0

model = celeri.build_model(config)

# Plot input data

In [ ]:
celeri.plot_input_summary(model)

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0 \\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0 \\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0 \\
    0                                                            & \mathrm{identities}                 & 0 \\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; rates}
\end{bmatrix}
$$

# Estimate block model parameters (dense)

In [ ]:
operators, estimation = celeri.assemble_and_solve_dense(model, tde=True, eigen=False)

# Plot model summary

In [ ]:
celeri.plot_estimation_summary(model, estimation)

# Start of synthetic slip distribution test

In [ ]:
def plot_mesh(mesh, slip_values, ax):
    """
    Plots a colored mesh where the x and y values correspond to longitude and depth respectively and
    and the color represents the slip value.

    Inputs:
        mesh: a mesh object
        slip_values: [float] an array specifying the `slip` value of each triangle
        ax: a matplotlib axis object

    Return:
        a PolyCollection object
    """
    # x_coords = mesh.meshio_object.points[:, 0]  # lon
    # y_coords = mesh.meshio_object.points[:, 1]  # lat
    x_coords = mesh.points[:, 0]
    y_coords = mesh.points[:, 1]
    vertex_array = np.asarray(mesh.verts)

    xy = np.c_[x_coords, y_coords]
    verts = xy[vertex_array]

    pc = collections.PolyCollection(
        verts,
        edgecolor="k",
        cmap="turbo",
        linewidth=0.1,
        alpha=1.0,
    )
    pc.set_array(slip_values)
    pc.set_clim([-30, 30])

    if not ax:
        ax = plt.gca()
    ax.add_collection(pc)
    ax.autoscale()
    return pc

In [ ]:
pc = plot_mesh(model.meshes[0], estimation.tde_dip_slip_rates, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

In [ ]:
def set_slip_values(mesh, slip_values, rect, fill_value):
    """
    Modifies the slip value of the mesh in a given rectangular section.

    Inputs:
        mesh: a mesh object
        slip_values: [float] an array specifying the `slip` value of each triangle
        rect: [[x0, y0], [x1, y1]] an array or tuple containing the lower and upper bounds (e.g. [28, 30])
        fill_value: float the value that will be used to override the mask's range

    Return:
        a slip array

    """
    [[lon0, depth0], [lon1, depth1]] = rect

    if depth0 > depth1:
        depth0, depth1 = depth1, depth0

    x_coords = mesh.points[:, 0]  # lon
    y_coords = mesh.points[:, 2]  # depth
    vertex_array = np.asarray(mesh.verts)
    xy = np.c_[x_coords, y_coords]
    verts = xy[vertex_array]
    n_triangles = len(vertex_array)

    for i in range(n_triangles):
        lon_centroid = mesh.centroids[i, 0]
        depth_centroid = mesh.centroids[i, 1]

        if lon0 <= lon_centroid < lon1 and depth0 <= depth_centroid < depth1:
            slip_values[i] = fill_value

    return slip_values


def checkerboard_slip_values(
    rng,
    mesh,
    lon_start,
    lon_end,
    lon_step,
    depth_start,
    depth_end,
    depth_step,
    slip_value_1,
    slip_value_2,
):
    """
    Creates a checkerboard pattern of the mesh for a given step in longitude and depth, and for a couple of slip values.

    Inputs:
        mesh: a mesh object
        lon_start: min longitude of the mesh
        lon_end: max longitude of the mesh
        lon_step: space covered by a rectangle along the lon axis -> (km/111)
        depth_start: min depth of the mesh
        depth_end: max depth of the mesh
        depth_step: space (in km) covered by a rectangle along the depth axis
        slip_value_1: [float] an array specifying the `slip` value of each triangle in a certain rectangle
        slip_value_1: [float] an array specifying the `slip` value of each triangle in the following rectangle

    Return:
        a slip array

    """
    n_triangles = len(mesh.verts)
    slip_values = rng.uniform(low=0, high=24, size=n_triangles)
    row = 0
    depth = depth_start

    while depth_start <= depth <= depth_end:
        col = 0
        lon = lon_start

        while lon_start <= lon <= lon_end:
            # select colors based on parity
            # slip_value = slip_value_1 if (row + col) % 2 == 0 else slip_value_2
            if (row + col) % 2 == 0:
                slip_value = slip_value_1
            else:
                slip_value = slip_value_2

            # set `slip_value` on current rectangle
            slip_values = set_slip_values(
                mesh,
                slip_values,
                [[lon, depth], [lon + lon_step, depth + depth_step]],
                slip_value,
            )

            # increment longitude iterators
            col += 1
            lon += lon_step

        # increment depth iterators
        row += 1
        depth += depth_step
    return slip_values


# Provide a seed for the random number generator
rng = np.random.default_rng(42)

slip_values = checkerboard_slip_values(
    rng,
    model.meshes[0],
    lon_start=model.meshes[0].x_perimeter.min(),
    lon_end=model.meshes[0].x_perimeter.max(),
    lon_step=2,
    depth_start=model.meshes[0].y_perimeter.min(),
    depth_end=model.meshes[0].y_perimeter.max(),
    depth_step=2,
    slip_value_1=30,
    slip_value_2=0,
)

pc = plot_mesh(model.meshes[0], slip_values, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

# Generate synthetic interseismic slip distribution on Cascadia

In [ ]:
# Replace inferred Cascadia rates with synthetic slip
synthetic_state_vector = np.copy(estimation.state_vector)
synthetic_data_vector = np.copy(estimation.data_vector)
# synthetic_state_vector[
#     3 * index.n_blocks + 1 : 3 * index.n_blocks + 2 * index.n_tde_total : 2
# ] = 0
# synthetic_state_vector[
#     3 * index.n_blocks : 3 * index.n_blocks + 2 * index.n_tde_total : 2
# ] = slip_values

synthetic_state_vector[
    3 * operators.index.n_blocks : 3 * operators.index.n_blocks
    + 2 * operators.index.n_tde_total : 2
] = 0
synthetic_state_vector[
    3 * operators.index.n_blocks + 1 : 3 * operators.index.n_blocks
    + 2 * operators.index.n_tde_total : 2
] = slip_values

# Forward velocities
synthetic_predictions = estimation.operator @ synthetic_state_vector
synthetic_data_vector[0 : 2 * operators.index.n_stations : 2] = synthetic_predictions[
    0 : 2 * operators.index.n_stations : 2
]

# Unbounded (classic) block model solve with synthetic slip deficit distribution

In [ ]:
estimated_synthetic_state_vector_unconstrained = (
    (estimation.state_covariance_matrix @ estimation.operator.T)
    * estimation.weighting_vector
    @ synthetic_data_vector
)

tde_rates_unconstrained = estimated_synthetic_state_vector_unconstrained[
    3 * operators.index.n_blocks : 3 * operators.index.n_blocks
    + 2 * operators.index.n_tde_total
]
tde_ss_rates_unconstrained = tde_rates_unconstrained[0::2]
tde_ds_rates_unconstrained = tde_rates_unconstrained[1::2]

pc = plot_mesh(model.meshes[0], tde_ds_rates_unconstrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

# Bounded (`lsq_linear`) block model solve with synthetic slip deficit distribution

In [ ]:
index = operators.index

lower_bound = np.zeros_like(estimation.state_vector)
upper_bound = np.zeros_like(estimation.state_vector)
lower_bound[:] = -np.inf
upper_bound[:] = np.inf

# Strike-slip
lower_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = -5
upper_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = 5

# Dip-slip
lower_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 0
upper_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 30

# Experiment: constrainted least squares
- The idea is to constrain TDE slip rates in some region

In [ ]:
lower_bound = np.zeros_like(estimation.state_vector)
upper_bound = np.zeros_like(estimation.state_vector)
lower_bound[:] = -np.inf
upper_bound[:] = np.inf

# Strike-slip
lower_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = -5
upper_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = 5

# Dip-slip
lower_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 0
upper_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 30

from scipy.optimize import lsq_linear

# Non-linear solver (bounded)
res = lsq_linear(
    estimation.operator * np.sqrt(estimation.weighting_vector[:, None]),
    synthetic_data_vector * np.sqrt(estimation.weighting_vector),
    bounds=(lower_bound, upper_bound),
    verbose=1,
)

In [ ]:
tde_rates_constrained = res.x[
    3 * index.n_blocks : 3 * index.n_blocks + 2 * index.n_tde_total
]
tde_ss_rates_constrained = tde_rates_constrained[0::2]
tde_ds_rates_constrained = tde_rates_constrained[1::2]

pc = plot_mesh(model.meshes[0], tde_ds_rates_constrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

In [ ]:
plt.figure(figsize=(12, 12))

plt.subplot(2, 3, 1)
plt.title("input strike-slip")
pc = plot_mesh(model.meshes[0], np.zeros_like(tde_ss_rates_unconstrained), plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

plt.subplot(2, 3, 2)
misfit = (
    np.sum(np.abs(tde_ss_rates_unconstrained - np.zeros_like(slip_values)))
    / model.meshes[0].n_tde
)
plt.title(f"constrained strike-slip \n (error: {misfit:.1f} mm/yr)")
pc = plot_mesh(model.meshes[0], tde_ss_rates_unconstrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

plt.subplot(2, 3, 3)
misfit = (
    np.sum(np.abs(tde_ss_rates_constrained - np.zeros_like(slip_values)))
    / model.meshes[0].n_tde
)
plt.title(f"constrained strike-slip \n (error: {misfit:.1f} mm/yr)")
pc = plot_mesh(model.meshes[0], tde_ss_rates_constrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")


plt.subplot(2, 3, 4)
plt.title("input dip-slip")
pc = plot_mesh(model.meshes[0], slip_values, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

plt.subplot(2, 3, 5)
misfit = (
    np.sum(np.abs(tde_ds_rates_unconstrained - slip_values)) / model.meshes[0].n_tde
)
plt.title(f"unconstrained dip-slip \n (error: {misfit:.1f} mm/yr)")
pc = plot_mesh(model.meshes[0], tde_ds_rates_unconstrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

plt.subplot(2, 3, 6)
misfit = np.sum(np.abs(tde_ds_rates_constrained - slip_values)) / model.meshes[0].n_tde
plt.title(f"constrained dip-slip \n (error: {misfit:.1f} mm/yr)")
pc = plot_mesh(model.meshes[0], tde_ds_rates_constrained, plt.gca())
plt.colorbar(pc)
plt.gca().set_facecolor("gainsboro")
plt.gca().set_aspect("equal")

# plt.savefig("cascadia_resolution.pdf")
plt.show()